In [43]:
import json
import csv
import re

In [44]:
def iterate_json(item, info_dict, current_key):
    if isinstance(item, list):
        for i,element in enumerate(item):
            iterate_json(element, info_dict, current_key + "." + str(i))
            
    elif isinstance(item, dict):
        for key in item:
            value = item[key]
            new_key = ""
            if current_key != "":
                new_key = current_key + "."
            new_key = new_key + str(key)
            iterate_json(value, info_dict, new_key)
    else:
        info_dict[current_key] = item
                

In [45]:
with open('pokedex.json') as jsonfile: 
    data = json.load(jsonfile)
    
with open('output.tsv', 'w', newline='') as output:
    tsv_writer = csv.writer(output, delimiter='\t')
    flat_data = []
    for data_point in data:
        per_item = {}
        iterate_json(data_point, per_item, "")
        flat_data.append(per_item)

    pattern_height = r'\((\d+\.\d+|\d+)m\)'
    pattern_weight = r'\((\d+\.\d+|\d+) kg\)'
    for row in flat_data:
        height = row.get('height', '')
        weight = row.get('weight', '')

        match = re.search(pattern_height, height)
        if match:
            metric_value = match.group(1) + 'm'
            row['height'] = metric_value
        else:
            row['height'] = None

        match = re.search(pattern_weight, weight)
        if match:
            metric_value = match.group(1) + ' kg'
            row['weight'] = metric_value
        else:
            row['weight'] = None

    headers = set()
    for item in flat_data:
        headers.update(item.keys())
    headers = sorted(headers)
    tsv_writer.writerow(headers)

    for row in flat_data:
        row_values = [row.get(header, '') for header in headers]
        tsv_writer.writerow(row_values)

with open('output.tsv', 'r') as tsvfile:
    reader = csv.DictReader(tsvfile, delimiter='\t')
    sorted_rows = sorted(reader, key=lambda row: row['name'], reverse=False)

with open('pokedex.tsv', 'w', newline='') as tsvfile:
    writer = csv.DictWriter(tsvfile,fieldnames=headers, delimiter='\t')
    writer.writeheader()
    writer.writerows(sorted_rows)